## requirements

In [58]:
import pandas as pd
import numpy as np
#pip install pandas-datareader
from pandas_datareader import data

from typing import List

## minimum variance model

The variance of a portfolio of financial assets will be minimized, yielding the percentage of ownership in each asset for a determined required level of return.

the algorithm is expressed below
$$ Min_{w_{i}}V_{p} \rightarrow L_{\left\{w_{i}\right\},\lambda_{1}\lambda_{2}}=\sum_{j=1}^n\sum_{j=1}^n w_{i}w_{j}+\lambda_{1}\left[ \sum_{i=1}^n w_{i}E\left(R_{i}\right)- E_{0}\right]+\lambda_{2}\left[ \sum_{i=1}^n w_{i}- 1\right] $$

$$ \frac{\partial L_{\left\{w_{i}\right\},\lambda_{1}\lambda_{2}}}{\partial w_{i}}=\sum_{j=1}^nw_{j}\sigma_{ij}+\lambda_{1}E\left(R_{i}\right)+\lambda_{2}=0 $$

$$ \frac{\partial L_{\left\{w_{i}\right\},\lambda_{1}\lambda_{2}}}{\partial \lambda_{1}}=\left[ \sum_{i=1}^n w_{i}E\left(R_{i}\right)- E_{0}\right]=0 $$

$$ \frac{\partial L_{\left\{w_{i}\right\},\lambda_{1}\lambda_{2}}}{\partial \lambda_{2}}=\left[ \sum_{i=1}^n w_{i}- 1\right] $$

$$ \begin{bmatrix}\left[2\sigma\right] & E\left(R\right) & 1 \\E\left(R\right) & 0 & 0 \\ 1 & 0 & 0\end{bmatrix}\begin{bmatrix}\left[W\right]  \\ \lambda_{1} \\ \lambda_{2} \end{bmatrix}=\begin{bmatrix}0  \\ E_{0} \\ 1 \end{bmatrix} $$

#### code

In [89]:
def rate(df):
    dfr = pd.DataFrame()
    for i in df.columns.values:
        dfr[f"{i}"] = df[f"{i}"] / df[f"{i}"].shift(1) - 1
    return dfr


def get_df(tikets:List[str],date:str)->dict:
    df = pd.DataFrame()
    invalid_tikets = []
    for i in tikets:
        try:
            df[f'{i}'] = data.DataReader(i,'yahoo',date)['Close']
        except:
            invalid_tikets.append(i)
    return {'dfr':rate(df),'df':df,'invalid_tikets':invalid_tikets}


In [107]:

def VM (R,dfr):
    vm_matrix=2*dfr.cov()
    vm_matrix["E(R)"]=dfr.mean()
    vm_matrix["w"]=1
    vm_matrix.loc[-2]=dfr.mean()
    vm_matrix.loc[-1]=1
    vm_matrix["w"][-2]=0
    vm_matrix["w"][-1]=0
    vm_matrix['E(R)'][-2]=0
    vm_matrix['E(R)'][-1]=0
    vm_matrix.rename(index={-2:'E(R)',-1:'w'}, inplace=True)
    vm_matrix_inv = pd.DataFrame(np.linalg.inv(vm_matrix.values), vm_matrix.columns, vm_matrix.index)
    vm_matrix_inv["w*"]=vm_matrix_inv["E(R)"]*(((1+R)**(1/252)-1))+vm_matrix_inv["w"]
    weight=vm_matrix_inv["w*"][:-2]
    return weight.to_dict()

In [90]:
close = get_df(['TSLA','GOOG','AAPL'],'2021-1-1')
VM(0.1,close['dfr'])